# Interoperability with Python: the `PyCall` package

Calling Python functions is  easy, thanks to the [`PyCall.jl` package](https://github.com/stevengj/PyCall.jl):

In [1]:
using PyCall

`PyCall` has a high-level interface that is designed so that the "transport" between Julia and Python is transparent from the user's point of view. For example, to import the Python `math` module, we do

In [2]:
@pyimport math

We can see what this actually does:

In [4]:
@macroexpand @pyimport math

quote  # /Users/dpsanders/.julia/v0.6/PyCall/src/PyCall.jl, line 466:
    if !((PyCall.isdefined)(:math)) # /Users/dpsanders/.julia/v0.6/PyCall/src/PyCall.jl, line 467:
        const math = (PyCall.pywrap)((PyCall.pyimport)("math"))
    else  # /Users/dpsanders/.julia/v0.6/PyCall/src/PyCall.jl, line 468:
        if !(math isa PyCall.Module) # /Users/dpsanders/.julia/v0.6/PyCall/src/PyCall.jl, line 469:
            (PyCall.error)("@pyimport: ", :math, " already defined")
        end
    end # /Users/dpsanders/.julia/v0.6/PyCall/src/PyCall.jl, line 471:
    PyCall.nothing
end

We can now mix and match Python calls, labelled by the `math.` qualifier, and Julia calls:

In [6]:
math.sin(0.3*math.pi) - sin(0.3*pi) 

-1.1102230246251565e-16

Array objects are automatically converted:

In [3]:
@pyimport numpy.random as nprandom
nprandom.rand(3,4)

3×4 Array{Float64,2}:
 0.184095  0.332699  0.594909   0.671562
 0.465193  0.316497  0.0114373  0.324822
 0.91203   0.696114  0.410632   0.708648

Let's define a Julia function:

In [4]:
objective(x) = cos(x) - x

objective (generic function with 1 method)

In [5]:
objective(3)

-3.989992496600445

We can pass this Julia function to a Python module:

In [7]:
using Conda
Conda.add("scipy")  # use Julia's Conda package to install a Python package

Solving environment: ...working... done




==> WARNING: A newer version of conda exists. <==
  current version: 4.4.10
  latest version: 4.5.0

Please update conda by running

    $ conda update -n base conda


scipy 1.0.0: #########9 |  99% 


## Package Plan ##

  environment location: /Users/dpsanders/.julia/v0.6/Conda/deps/usr

  added / updated specs: 
    - scipy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    scipy-1.0.0                |py27_blas_openblas_201        15.7 MB  conda-forge

The following NEW packages will be INSTALLED:

    scipy: 1.0.0-py27_blas_openblas_201 conda-forge [blas_openblas]


Preparing transaction: ...working... 

scipy 1.0.0: ########## | 100% 


done
Verifying transaction: ...working... done
Executing transaction: ...working... done


In [6]:
@pyimport scipy.optimize as so
so.newton(objective, 1)

LoadError: [91mPyError (ccall(@pysym(:PyImport_ImportModule), PyPtr, (Cstring,), name)

The Python package scipy.optimize could not be found by pyimport. Usually this means
that you did not install scipy.optimize in the Python version being used by PyCall.

PyCall is currently configured to use the Julia-specific Python distribution
installed by the Conda.jl package.  To install the scipy.optimize module, you can
use `pyimport_conda("scipy.optimize", PKG)`, where PKG is the Anaconda
package the contains the module scipy.optimize, or alternatively you can use the
Conda package directly (via `using Conda` followed by `Conda.add` etcetera).

Alternatively, if you want to use a different Python distribution on your
system, such as a system-wide Python (as opposed to the Julia-specific Python),
you can re-configure PyCall with that Python.   As explained in the PyCall
documentation, set ENV["PYTHON"] to the path/name of the python executable
you want to use, run Pkg.build("PyCall"), and re

## Accessing fields and methods of Python objects

Accessing fields (properties) and methods of Python objects uses the `obj.a` and `obj.b()` syntax, where `obj` is a Python object.

In Julia 0.6 the `obj.b` syntax in Julia is restricted to accessing fields of Julia composite types, so to access fields and methods of Python objects via `PyCall.jl`, it is necessary to use the syntax

`obj[:a]` for fields, and

`obj[:a]()` for methods

Here, the Julia syntax `:a` refers to the Julia symbol `a`.

In 0.7, it will be possible to use the `obj.b` syntax.

# Interoperability with C

Julia has a simple way to call C and Fortran functions in shared libraries, via the `ccall` function.

In [9]:
?ccall

search: ccall AbstractChannel



```
ccall((symbol, library) or function_pointer, ReturnType, (ArgumentType1, ...), ArgumentValue1, ...)
```

Call function in C-exported shared library, specified by `(function name, library)` tuple, where each component is a string or symbol.

Note that the argument type tuple must be a literal tuple, and not a tuple-valued variable or expression. Alternatively, `ccall` may also be used to call a function pointer, such as one returned by `dlsym`.

Each `ArgumentValue` to the `ccall` will be converted to the corresponding `ArgumentType`, by automatic insertion of calls to `unsafe_convert(ArgumentType, cconvert(ArgumentType, ArgumentValue))`. (See also the documentation for each of these functions for further details.) In most cases, this simply results in a call to `convert(ArgumentType, ArgumentValue)`.


We see that we must specify:

- the name of the function, as a Julia symbol or as a string
- the name of the shared library where it lives;  these are given as an ordered pair (tuple)

- the return type of the function

- the argument types that the function accepts, as a tuple

- the arguments themselves

A simple example is to call the `sin` function from the `libm` (math) library:

In [14]:
x = ccall( (:sin, "libm"), Float64, (Float64,), 0.1)

0.09983341664682815

In [15]:
sin(0.1)

0.09983341664682815

Note that a tuple with a single element is written with a single trailing `,`.

## Wrapping your own C code

The following example of wrapping your own C code is taken from a short course by Steven Johnson that is highly recommended: https://github.com/stevengj/18S096 (lecture 1):

In [19]:
C_code = """
#include <stddef.h>
double c_sum(size_t n, double *X) {
    double s = 0.0;
    for (size_t i = 0; i < n; ++i) {
        s += X[i];
    }
    return s;
}
"""

# compile to a shared library by piping C_code to gcc
# (only works if you have gcc installed):

const Clib = tempname()  # generate a temporary file

open(`gcc -fPIC -O3 -msse3 -xc -shared -o $(Clib * "." * Libdl.dlext) -`, "w") do f
    print(f, C_code)
end

c_sum(X::Array{Float64}) = ccall(("c_sum", Clib), Float64, (Csize_t, Ptr{Float64}), length(X), X)

c_sum (generic function with 1 method)

In [17]:
Libdl.dlext   # specifies the correct termination for a shared library on the current system

"dylib"

In [20]:
Clib

"/var/folders/h0/68t1xc991mq20mrkn6yy6n5h0000gn/T/juliadpiXdk"

The (strange, in my opinion) [`do` block syntax](https://docs.julialang.org/en/stable/manual/functions/#Do-Block-Syntax-for-Function-Arguments-1) creates an anonymous function from the body of the `do` block, and passes the result of the `open` call in as the argument `f`.

The code between backticks ( \` ) creates a command object. The `$(...)` interpolates the result of running that piece of Julia code into the command object.

## Wrapping C code in a package

Usually we wrap C code that will not change, so it is more common to compile the C code once and for all to a shared library and wrap it all in a package.

For an example of how to do this in a package, see the author's [`CRlibm.jl` package](https://github.com/dpsanders/CRlibm.jl). 

There the C code is compiled when the package is installed, via the `deps/build.jl` script, which creates the `libcrlibm` shared library in a known location inside the package.